In [45]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import xgboost as xgb
# check version number
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import NearMiss
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from bayes_opt import BayesianOptimization

In [2]:
train = pd.read_csv('../../Data/Processed/train.csv')
val = pd.read_csv('../../Data/Processed/val.csv')
test = pd.read_csv('../../Data/Processed/test.csv')


In [3]:
test = test.set_index('SK_ID_CURR')

In [5]:
train = train.set_index('SK_ID_CURR')
val = val.set_index('SK_ID_CURR')

In [78]:
full  = pd.concat([train, val], axis = 0)

,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,REGION_RATING_CLIENT,AMT_INCOME_TOTAL,AMT_CREDIT,...,AMT_CREDIT_SUM_nonrev,AMT_CREDIT_SUM_DEBT_nonrev,AMT_CREDIT_SUM_LIMIT_nonrev,AMT_CREDIT_SUM_OVERDUE_nonrev,AMT_ANNUITY_nonrev,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,SK_DPD,SK_DPD_DEF
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,0,1,0,1,0,0,2,0.409751,-0.475080,...,-0.287756,-0.138381,0.018811,-0.028467,-0.217978,0.425771,-0.12526,-0.454485,-1.0,-1.0
100003,1,0,1,0,1,0,0,1,1.180915,1.925097,...,-0.599048,-0.470529,0.018811,-0.028467,-0.217978,0.425771,-0.12526,-0.454485,-1.0,-1.0
100004,1,1,1,0,0,0,0,2,-1.132578,-1.210088,...,-0.552526,-0.470529,0.018811,-0.028467,-0.217978,0.425771,-0.12526,-0.454485,-1.0,-1.0
100006,1,0,0,0,1,0,0,2,-0.361414,-0.729237,...,-0.645947,-0.470530,-0.097396,-0.030343,-0.218001,0.220099,-0.12522,1.282625,0.0,0.0
100007,1,0,0,0,0,0,1,2,-0.515647,-0.187130,...,-0.573672,-0.470529,0.018811,-0.028467,-0.217978,0.425771,-0.12526,-0.454485,-1.0,-1.0


In [13]:
y_train = train['TARGET']
y_test = val['TARGET']
cols = list(train.columns)
cols.remove('TARGET')
X_train = train[cols]
X_test = val[cols]


In [70]:
def grid_search(model, params, X, y):
    
    cv = GridSearchCV(model, params, cv = 3)
    cv.fit(X, y)
    
    return cv.best_params_

def train_model(model, X, y):
    model.fit(X, y)
    y_pred = model.predict(X)
    
    return model

def get_scores(model, X, y):
    y_pred = model.predict(X)
    print(classification_report(y, y_pred))

def get_output(model, X):
    
    ypred =model.predict_proba(X)
    
    return ypred

def get_auc(model, X, y):
    
    
    y_pred = get_output(model, X)
    return roc_auc_score(y, y_pred[:,1])

    

In [15]:
clf = RandomForestClassifier()
params = {'max_depth' : [25, 35], 'min_samples_split' : [100], 'criterion' : ['gini'], 'n_estimators' : [1000, 750], 'class_weight' : ['balanced']}
best_params = grid_search(clf, params, X_train, y_train)
print("Best Params are:", best_params)
clf = RandomForestClassifier(**best_params)

clf = train_model(clf, X_train, y_train)

print("Training Scores")
get_scores(clf, X_train, y_train)

print("Testing Scores")
get_scores(clf, X_test, y_test)

Best Params are: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 35, 'min_samples_split': 100, 'n_estimators': 1000}
Training Scores
              precision    recall  f1-score   support

           0       0.98      0.91      0.95    226148
           1       0.45      0.81      0.58     19860

    accuracy                           0.91    246008
   macro avg       0.72      0.86      0.76    246008
weighted avg       0.94      0.91      0.92    246008

Testing Scores
              precision    recall  f1-score   support

           0       0.94      0.90      0.92     56538
           1       0.24      0.38      0.30      4965

    accuracy                           0.86     61503
   macro avg       0.59      0.64      0.61     61503
weighted avg       0.89      0.86      0.87     61503



In [14]:
get_scores(clf, X_test, y_test)

              precision    recall  f1-score   support

           0       0.94      0.89      0.92     56538
           1       0.24      0.40      0.30      4965

    accuracy                           0.85     61503
   macro avg       0.59      0.64      0.61     61503
weighted avg       0.89      0.85      0.87     61503



In [12]:
y_test.shape

(61503,)

In [42]:
lgbm = LGBMClassifier(boosting_type='dart',  objective='binary', num_boost_round=2000, learning_rate=0.05, metric='auc', max_depth = 20, n_estimators = 10000, subsample=0.8715623, num_leaves=34, min_split_gain=0.0222415,
params = {'class_weight' : ['balanced'], 'reg_alpha' : [1]}
best_params = grid_search(lgbm, params, X_train, y_train)
print("Best Params are:", best_params)
lgbm = LGBMClassifier(**best_params)

lgbm = train_model(lgbm, X_train, y_train)

print("Training Scores")
get_scores(lgbm, X_train, y_train)

print("Testing Scores")
get_scores(lgbm, X_test, y_test)

C:\Users\siyer\anaconda3\lib\site-packages\lightgbm\engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] num_iterations is set=2000, num_boost_round=2000 will be ignored. Current value: num_iterations=2000
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leav

In [41]:
get_auc(lgbm, X_test, y_test)

ROC 0.7609485579074036


In [27]:
columns= [i.replace(':', '') for i in list(X_train.columns)]
columns= [i.replace(' ', '') for i in columns]


In [28]:
X_train.columns = columns
X_test.columns = columns


In [67]:
def objective(n_estimators, learning_rate, num_leaves, colsample_bytree, subsample,max_depth, reg_alpha, reg_lambda, min_split_gain, min_child_weight):
    
    #lgbm.fit(X_train, y_train)
    #print(**hyperparameters)
    
    params = {
        'n_estimators': int(n_estimators) , 
        'learning_rate': learning_rate, 
        'num_leaves' : int(num_leaves), 
        'colsample_bytree' : colsample_bytree, 
        'subsample' : subsample,
        'max_depth' : int(max_depth), 
        'reg_alpha' : reg_alpha,
        'reg_lambda' : reg_lambda,
        'min_split_gain' :min_split_gain,
        'min_child_weight': min_child_weight
    }
    
    lgbm = LGBMClassifier(**params)
    lgbm.fit(X_train, y_train)

    return get_auc(lgbm, X_test, y_test)

In [59]:
pbounds = {'n_estimators': (1000, 10000), 'learning_rate': (0.01, 0.6), 'num_leaves' : (10, 100), 'colsample_bytree' : (0.7, 1), 'subsample' : (0.7, 1), 'max_depth' : (6, 50), 'reg_alpha' : (0, 1), 'reg_lambda' : (0, 1), 'min_split_gain' : (0, 1),'min_child_weight': (0, 1000)}

In [60]:
pbounds


{'n_estimators': (1000, 10000),
 'learning_rate': (0.01, 0.6),
 'num_leaves': (10, 100),
 'colsample_bytree': (0.7, 1),
 'subsample': (0.7, 1),
 'max_depth': (6, 50),
 'reg_alpha': (0, 1),
 'reg_lambda': (0, 1),
 'min_split_gain': (0, 1),
 'min_child_weight': (0, 1000)}

In [72]:

optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=1,
)

In [74]:
optimizer.maximize(
    init_points=5,
    n_iter=1000,
)

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | min_sp... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  106      |  0.7574   |  0.7058   |  0.4105   |  15.31    |  265.5    |  0.4916   |  1.48e+03 |  61.67    |  0.1467   |  0.5893   |  0.9099   |
|  107      |  0.7341   |  0.7307   |  0.2543   |  36.55    |  414.2    |  0.04995  |  5.823e+0 |  69.74    |  0.5149   |  0.9446   |  0.876    |
|  108      |  0.7664   |  0.971    |  0.09111  |  12.13    |  807.4    |  0.3977   |  2.488e+0 |  93.48    |  0.3478   |  0.7508   |  0.9178   |
|  109      |  0.7534   |  0.965    |  0.378    |  39.04    |  348.9    |  0.2699   |  9.063e+0 |  48.53    |  0.9648   |  0.6634   |  0.8865   |
|  110      |  0.7529   |  0.7344   |  0.5702   |  25.8     |  578.4    |  0.4081   |  3.133e+0 |  91.3     |  0.5737   |  0

|  139      |  0.7555   |  0.8323   |  0.5733   |  39.93    |  542.7    |  0.7766   |  3.876e+0 |  32.93    |  0.7771   |  0.01974  |  0.927    |
|  140      |  0.7656   |  0.754    |  0.08001  |  11.79    |  498.1    |  0.3798   |  3.83e+03 |  44.78    |  0.576    |  0.3719   |  0.8978   |
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
|  141      |  0.7433   |  0.9448   |  0.5048   |  31.16    |  515.1    |  0.1637   |  3.832e+0 |  31.11    |  0.5651   |  0.1604   |  0.7826   |
|  142      |  0.7633   |  0.7976   |  0.3002   |  11.25    |  474.0    |  0.7061   |  3.835e+0 |  62.22    |  0.6454   |  0.905    |  0.8922   |
|  143      |  0.7529   |  0.9744   |  0.5851   |  45.05    |  775.9    |  0.2542   |  2.471e+0 |  20.45    |  0.7838   |  0.6586   |  0.8415   |
|  144      |  0.7585   |  0.7905   |  0.5617   |  44.16    |  838.8    |  0.5239   |  1.041e+0 |  90.47    |  0.07653  |  0.1216   |  0.996

|  173      |  0.7656   |  0.9212   |  0.01838  |  8.189    |  859.4    |  0.257    |  2.579e+0 |  27.66    |  0.9142   |  0.2707   |  0.8717   |
|  174      |  0.764    |  0.9894   |  0.2102   |  29.08    |  861.2    |  0.262    |  2.591e+0 |  44.76    |  0.4567   |  0.7547   |  0.8702   |
|  175      |  0.7632   |  0.9368   |  0.1367   |  46.47    |  727.5    |  0.9881   |  2.525e+0 |  14.48    |  0.137    |  0.8244   |  0.8254   |
|  176      |  0.7619   |  0.7768   |  0.4823   |  44.22    |  769.5    |  0.6735   |  2.517e+0 |  10.17    |  0.972    |  0.553    |  0.8754   |
|  177      |  0.7642   |  0.8024   |  0.07221  |  11.31    |  763.7    |  0.8575   |  1.015e+0 |  89.18    |  0.7253   |  0.922    |  0.9208   |
|  178      |  0.7595   |  0.9433   |  0.5406   |  9.041    |  882.0    |  0.3474   |  2.578e+0 |  38.39    |  0.5818   |  0.6093   |  0.8649   |
|  179      |  0.7602   |  0.9131   |  0.5148   |  11.57    |  823.5    |  0.4074   |  1.044e+0 |  75.91    |  0.922    |  0

|  207      |  0.7656   |  0.7587   |  0.1097   |  13.36    |  551.5    |  0.4713   |  3.898e+0 |  19.86    |  0.08103  |  0.05001  |  0.8331   |
|  208      |  0.7666   |  0.8689   |  0.0163   |  25.27    |  651.5    |  0.1196   |  2.511e+0 |  13.29    |  0.6981   |  0.7985   |  0.7861   |
|  209      |  0.7574   |  0.9815   |  0.5243   |  7.528    |  512.6    |  0.3308   |  3.854e+0 |  40.52    |  0.9008   |  0.08666  |  0.9678   |
|  210      |  0.764    |  0.9385   |  0.07924  |  8.033    |  583.8    |  0.9587   |  3.881e+0 |  99.6     |  0.3164   |  0.7581   |  0.7044   |
|  211      |  0.7617   |  0.8362   |  0.1488   |  37.23    |  651.1    |  0.1397   |  2.516e+0 |  43.04    |  0.9803   |  0.7885   |  0.9489   |
|  212      |  0.7595   |  0.9879   |  0.5565   |  32.83    |  558.8    |  0.31     |  3.889e+0 |  10.7     |  0.8876   |  0.5564   |  0.943    |
|  213      |  0.7642   |  0.7712   |  0.2406   |  17.23    |  769.0    |  0.5672   |  1.029e+0 |  80.61    |  0.9677   |  0

|  241      |  0.7642   |  0.8073   |  0.1364   |  9.55     |  854.1    |  0.1428   |  2.584e+0 |  50.07    |  0.5604   |  0.3487   |  0.9347   |
|  242      |  0.7554   |  0.7979   |  0.4716   |  14.89    |  858.8    |  0.2484   |  2.564e+0 |  57.26    |  0.5011   |  0.6251   |  0.8545   |
|  243      |  0.7666   |  0.7386   |  0.03216  |  42.46    |  681.1    |  0.341    |  2.529e+0 |  43.54    |  0.7656   |  0.06058  |  0.8109   |
|  244      |  0.7667   |  0.7492   |  0.01823  |  20.0     |  617.5    |  0.366    |  2.592e+0 |  86.09    |  0.2567   |  0.4543   |  0.9064   |
|  245      |  0.7625   |  0.919    |  0.3145   |  22.27    |  770.8    |  0.7272   |  1.006e+0 |  91.23    |  0.4022   |  0.1581   |  0.7801   |
|  246      |  0.7591   |  0.7856   |  0.5007   |  42.75    |  793.0    |  0.9102   |  1.037e+0 |  57.49    |  0.6519   |  0.9549   |  0.8724   |
|  247      |  0.7591   |  0.8788   |  0.463    |  10.2     |  662.3    |  0.9163   |  2.68e+03 |  95.37    |  0.9847   |  0

...

|  954      |  0.7359   |  0.7975   |  0.5525   |  24.85    |  490.7    |  0.156    |  1.478e+0 |  40.74    |  0.7698   |  0.0615   |  0.9487   |
|  955      |  0.7636   |  0.9965   |  0.3093   |  33.1     |  923.9    |  0.4935   |  5.431e+0 |  85.91    |  0.499    |  0.7727   |  0.7077   |
|  956      |  0.7619   |  0.9212   |  0.2478   |  29.74    |  146.3    |  0.6824   |  9.332e+0 |  54.98    |  0.2766   |  0.8193   |  0.8607   |
|  957      |  0.759    |  0.9162   |  0.4973   |  27.84    |  912.0    |  0.3325   |  8.289e+0 |  65.89    |  0.04789  |  0.5851   |  0.8097   |
|  958      |  0.752    |  0.8755   |  0.5542   |  44.26    |  262.7    |  0.4557   |  3.678e+0 |  30.19    |  0.5281   |  0.2784   |  0.9833   |
|  959      |  0.7535   |  0.8964   |  0.1825   |  41.66    |  208.0    |  0.1798   |  5.86e+03 |  85.68    |  0.6618   |  0.3674   |  0.8646   |
|  960      |  0.7599   |  0.8461   |  0.4463   |  18.28    |  875.9    |  0.4958   |  7.326e+0 |  54.06    |  0.3473   |  0

|  988      |  0.7453   |  0.9978   |  0.4764   |  31.79    |  286.8    |  0.2518   |  9.797e+0 |  34.1     |  0.9563   |  0.9555   |  0.8766   |
|  989      |  0.7655   |  0.8686   |  0.076    |  10.48    |  473.5    |  0.3032   |  3.337e+0 |  45.6     |  0.6306   |  0.2885   |  0.9955   |
|  990      |  0.7632   |  0.97     |  0.23     |  49.73    |  760.6    |  0.8677   |  2.51e+03 |  41.56    |  0.6113   |  0.9637   |  0.7066   |
|  991      |  0.7664   |  0.7876   |  0.0343   |  36.86    |  212.5    |  0.2882   |  9.102e+0 |  11.37    |  0.7107   |  0.9279   |  0.8778   |
|  992      |  0.7636   |  0.961    |  0.1435   |  9.322    |  26.04    |  0.8669   |  8.072e+0 |  40.05    |  0.974    |  0.9364   |  0.9053   |
|  993      |  0.7619   |  0.7058   |  0.2505   |  11.77    |  342.6    |  0.436    |  5.656e+0 |  42.42    |  0.6909   |  0.7287   |  0.9067   |
|  994      |  0.7618   |  0.9169   |  0.3282   |  27.33    |  570.4    |  0.4179   |  5.586e+0 |  35.29    |  0.1722   |  0

|  1022     |  0.7661   |  0.7694   |  0.05945  |  17.05    |  807.8    |  0.3849   |  1.504e+0 |  68.56    |  0.6086   |  0.0294   |  0.9343   |
|  1023     |  0.7629   |  0.7373   |  0.2674   |  19.44    |  326.1    |  0.8895   |  8.409e+0 |  25.07    |  0.7778   |  0.6247   |  0.8328   |
|  1024     |  0.7248   |  0.9171   |  0.1635   |  49.42    |  30.96    |  0.01448  |  4.919e+0 |  56.85    |  0.381    |  0.7542   |  0.7952   |
|  1025     |  0.7608   |  0.9303   |  0.4023   |  30.32    |  778.1    |  0.4738   |  6.076e+0 |  73.64    |  0.6922   |  0.6083   |  0.9073   |
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
|  1026     |  0.7641   |  0.8745   |  0.141    |  42.37    |  709.1    |  0.7554   |  2.609e+0 |  31.35    |  0.4637   |  0.5636   |  0.8244   |
|  1027     |  0.7628   |  0.9088   |  0.2223   |  8.088    |  958.5    |  0.6032   |  9.609e+0 |  37.55    |  0.09397  |  0.3508   |  0.965

|  1056     |  0.7643   |  0.8496   |  0.05401  |  26.98    |  878.9    |  0.725    |  2.149e+0 |  23.85    |  0.7173   |  0.3505   |  0.9674   |
|  1057     |  0.7523   |  0.7959   |  0.4085   |  14.13    |  302.3    |  0.3268   |  8.742e+0 |  72.4     |  0.03159  |  0.691    |  0.8778   |
|  1058     |  0.7514   |  0.8622   |  0.4219   |  40.64    |  162.1    |  0.6771   |  9.32e+03 |  85.94    |  0.4129   |  0.2393   |  0.9262   |
|  1059     |  0.7649   |  0.9282   |  0.1363   |  14.85    |  902.2    |  0.621    |  4.628e+0 |  13.94    |  0.8021   |  0.6366   |  0.725    |
|  1060     |  0.7587   |  0.7927   |  0.5075   |  43.47    |  698.4    |  0.3331   |  2.6e+03  |  12.11    |  0.9025   |  0.1511   |  0.9531   |
|  1061     |  0.7546   |  0.8545   |  0.5778   |  6.461    |  260.2    |  0.5781   |  9.906e+0 |  87.67    |  0.6991   |  0.9742   |  0.9208   |
|  1062     |  0.7611   |  0.8835   |  0.3787   |  6.616    |  139.9    |  0.6559   |  3.156e+0 |  79.46    |  0.8575   |  0

|  1090     |  0.7354   |  0.7329   |  0.171    |  40.74    |  114.7    |  0.06117  |  7.862e+0 |  76.08    |  0.6804   |  0.5766   |  0.9602   |
|  1091     |  0.7624   |  0.8064   |  0.2612   |  29.66    |  936.2    |  0.486    |  3.106e+0 |  20.71    |  0.6613   |  0.9076   |  0.9406   |
|  1092     |  0.7525   |  0.759    |  0.2944   |  26.41    |  819.1    |  0.1037   |  2.53e+03 |  73.39    |  0.1501   |  0.553    |  0.7204   |
|  1093     |  0.7578   |  0.8049   |  0.3543   |  19.16    |  662.7    |  0.1901   |  2.683e+0 |  82.12    |  0.2577   |  0.7153   |  0.7039   |
|  1094     |  0.7307   |  0.8494   |  0.2888   |  38.18    |  164.4    |  0.1123   |  6.409e+0 |  76.54    |  0.5199   |  0.1462   |  0.7288   |
|  1095     |  0.7597   |  0.7006   |  0.5508   |  7.428    |  865.1    |  0.6534   |  9.238e+0 |  45.52    |  0.5092   |  0.6282   |  0.7185   |
|  1096     |  0.7539   |  0.8873   |  0.5599   |  23.26    |  234.0    |  0.6858   |  1.613e+0 |  34.71    |  0.663    |  0

In [76]:
optimizer.max

{'target': 0.7672007387522201,
 'params': {'colsample_bytree': 0.7553866957097399,
  'learning_rate': 0.01270335279588613,
  'max_depth': 33.06964203752187,
  'min_child_weight': 54.361067619027594,
  'min_split_gain': 0.47460637384049875,
  'n_estimators': 5482.338499321809,
  'num_leaves': 16.477123948540417,
  'reg_alpha': 0.8791643154086911,
  'reg_lambda': 0.8163341639018706,
  'subsample': 0.8747785752316152}}

In [82]:
best_params = {'colsample_bytree': 0.7553866957097399,
  'learning_rate': 0.01270335279588613,
  'max_depth': 33,
  'min_child_weight': 54.361067619027594,
  'min_split_gain': 0.47460637384049875,
  'n_estimators': 5482,
  'num_leaves': 16,
  'reg_alpha': 0.8791643154086911,
  'reg_lambda': 0.8163341639018706,
  'subsample': 0.8747785752316152}

In [88]:
y = full['TARGET']
cols = list(train.columns)
cols.remove('TARGET')
X = full[cols]
X.columns = X_train.columns


In [89]:
lgbm = LGBMClassifier(**best_params)
lgbm.fit(X,y)


LGBMClassifier(colsample_bytree=0.7553866957097399,
               learning_rate=0.01270335279588613, max_depth=33,
               min_child_weight=54.361067619027594,
               min_split_gain=0.47460637384049875, n_estimators=5482,
               num_leaves=16, reg_alpha=0.8791643154086911,
               reg_lambda=0.8163341639018706, subsample=0.8747785752316152)

In [105]:
params = {'colsample_bytree': 0.7553866957097399,
  'learning_rate': 0.01270335279588613,
  'max_depth': 36,
  'min_child_weight': 54.361067619027594,
  'min_split_gain': 0.47460637384049875,
  'n_estimators': 6000,
  'num_leaves': 16,
  'reg_alpha': 0.8791643154086911,
  'reg_lambda': 0.8163341639018706,
  'subsample': 0.8747785752316152}

In [106]:
lgbm = LGBMClassifier(**params)
lgbm.fit(X,y)


LGBMClassifier(colsample_bytree=0.7553866957097399,
               learning_rate=0.01270335279588613, max_depth=36,
               min_child_weight=54.361067619027594,
               min_split_gain=0.47460637384049875, n_estimators=6000,
               num_leaves=16, reg_alpha=0.8791643154086911,
               reg_lambda=0.8163341639018706, subsample=0.8747785752316152)

In [107]:
pred = get_output(lgbm, test)

In [108]:
pd.DataFrame(pred[:,1], index =  test.index, columns = ['TARGET']).to_csv('lightgbm_sub_1.csv')

In [93]:
test.drop(columns = 'Unnamed: 0', inplace = True )